In [ ]:
!pip install -q pandas

In [ ]:
import os
import torch
import pandas as pd

In [ ]:
def measure_data(measure_path):
  measure = torch.load(measure_path)
  val_losses = measure["val_losses"]
  train_scores = measure["train_scores"]
  metric_names = ["map", "map_50", "map_75", "map_medium", "map_large", "mar_1", "mar_10", "mar_100", "mar_medium", "mar_large"]
  maps, map_50s, map_75s, map_mediums, map_larges, mar_1s, mar_10s, mar_100s, mar_mediums, mar_larges = \
    zip(*[[score.item() for name, score in train_score.items() if name in metric_names] for train_score in train_scores])
  return {
    "val_loss": val_losses,
    "map": maps,
    "map_50": map_50s,
    "map_75": map_75s,
    "map_medium": map_mediums,
    "map_large": map_larges,
    "mar_1": mar_1s,
    "mar_10": mar_10s,
    "mar_100": mar_100s,
    "mar_medium": mar_mediums,
    "mar_large": mar_larges,
  }

In [ ]:
for measure_file in os.listdir("measures"):
  filename = measure_file.split(".")[0]
  measure_path = os.path.join("measures", measure_file)
  data = measure_data(measure_path)
  df = pd.DataFrame(data)
  df.to_csv(f"{filename}.csv", index=False)

In [ ]:
def train_data(last_checkpoint_path):
  last_checkpoint = torch.load(last_checkpoint_path, map_location="cuda")
    
  train_losses = last_checkpoint["train_losses"]
  val_scores = last_checkpoint["scores"]
    
  metric_names = ["map", "map_50", "map_75", "map_medium", "map_large", "mar_1", "mar_10", "mar_100", "mar_medium", "mar_large"]
  maps, map_50s, map_75s, map_mediums, map_larges, mar_1s, mar_10s, mar_100s, mar_mediums, mar_larges = \
    zip(*[[score.item() for name, score in val_score.items() if name in metric_names] for val_score in val_scores])
    
  return {
    "train/loss": train_losses,
    "val/map": maps,
    "val/map_50": map_50s,
    "val/map_75": map_75s,
    "val/map_medium": map_mediums,
    "val/map_large": map_larges,
    "val/mar_1": mar_1s,
    "val/mar_10": mar_10s,
    "val/mar_100": mar_100s,
    "val/mar_medium": mar_mediums,
    "val/mar_large": mar_larges,
  }

In [ ]:
import os
import torch
import pandas as pd
import json

models = ["fasterrcnn", "retinanet"]
parts = ["full", "head", "split-b1e-04-h1e-02"]

dirs = [f"{model}-{part}" for model in models for part in parts]

for dir in dirs:
  if dir == "fasterrcnn-split-b1e-04-h1e-02":
    continue

  checkpoints_path = os.path.join(dir, "checkpoints.json")
  with open(checkpoints_path, "r") as f:
    checkpoints = {int(epoch): path for epoch, path in json.load(f).items()}

  last_checkpoint_path = checkpoints[max(checkpoints.keys())]

  data = train_data(last_checkpoint_path)
  df = pd.DataFrame(data)
  df.to_csv(f"{dir}.csv", index=False)

In [ ]:
for measure_file in os.listdir("measures"):
  if "csv" not in measure_file:
    continue
  df: pd.DataFrame = pd.read_csv(os.path.join("measures", measure_file))
  data = {}
  for column in df.columns:
    if column == "val_loss":
      data["val/loss"] = df["val_loss"]
    else:
      data["train/" + column] = df[column]
  pd.DataFrame(data).to_csv(measure_file, index=False)

In [ ]:
logs1 = ["fasterrcnn-full", "fasterrcnn-head", "retinanet-full", "retinanet-head", "retinanet-split-b1e-04-h1e-02"]
logs2 = [log + "-measure" for log in logs1]

In [ ]:
csvs1 = [log + ".csv" for log in logs1]
csvs2 = [log + ".csv" for log in logs2]

In [ ]:
for csv1, csv2 in zip(csvs1, csvs2):
  df1 = pd.read_csv(csv1)
  df2 = pd.read_csv(csv2)

  columns1 = df1.columns.tolist()
  columns2 = df2.columns.tolist()

  data = {}

  for column in sorted(columns1 + columns2):
    data[column] = df1[column] if column in columns1 else df2[column]

  filename = csv1.split(".")[0] + "-all.csv"
  pd.DataFrame(data).to_csv(filename, index=False)